In [1]:
import heapq
import os
import re

import aocd
import numpy as np
from IPython.display import HTML
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2025, day=4)

In [3]:
desc = "".join(p._get_prose().split("---")[1:])
HTML(desc);

In [4]:
p.examples[0].input_data.split("\n")

['..@@.@@@@.',
 '@@@.@.@.@@',
 '@@@@@.@.@@',
 '@.@@@@..@.',
 '@@.@@@@.@@',
 '.@@@@@@@.@',
 '.@.@.@.@@@',
 '@.@@@.@@@@',
 '.@@@@@@@@.',
 '@.@.@@@.@.']

In [5]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data

    data = data.split("\n")
    return np.array([[c for c in line] for line in data])

## Trad looping approach

In [6]:
data = get_data()

In [7]:
directions = [(1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]

In [8]:
def get_pos_char(data, pos):
    if pos[0] < 0 or pos[0] >= data.shape[0]:
        return None
    if pos[1] < 0 or pos[1] >= data.shape[1]:
        return None
    return data[pos[0], pos[1]]

In [9]:
%%time
# Part 1

res = 0

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i, j] == "@":
            found = 0
            for d in directions:
                ni, nj = i + d[0], j + d[1]
                nc = get_pos_char(data, (ni, nj))
                if nc == "@":
                    found += 1
            if found < 4:
                res += 1

res

CPU times: user 49 ms, sys: 758 μs, total: 49.7 ms
Wall time: 49.1 ms


1493

In [10]:
%%time
# Part 2

res = 0

round_res = 1

while round_res > 0:
    new_data = data.copy()
    round_res = 0

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i, j] == "@":
                found = 0
                for d in directions:
                    ni, nj = i + d[0], j + d[1]
                    nc = get_pos_char(data, (ni, nj))
                    if nc == "@":
                        found += 1
                if found < 4:
                    round_res += 1
                    new_data[i, j] = "."
    res += round_res
    data = new_data

res

CPU times: user 976 ms, sys: 2.08 ms, total: 978 ms
Wall time: 978 ms


9194

## Optimised convolution approach

In [11]:
kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

data = get_data()

In [12]:
%%time
# Part 1

mask = (data == "@").astype(int)
neighbor_count = convolve(mask, kernel, mode="constant", cval=0)

res = np.sum((mask == 1) & (neighbor_count < 4))
res

CPU times: user 777 μs, sys: 1.01 ms, total: 1.78 ms
Wall time: 820 μs


1493

In [13]:
%%time
# Part 2

mask = data == "@"  # Work with boolean array directly

res = 0
round_res = 1

while round_res > 0:
    neighbor_count = convolve(mask.astype(int), kernel, mode="constant", cval=0)
    to_remove = mask & (neighbor_count < 4)
    round_res = np.sum(to_remove)

    mask[to_remove] = False
    res += round_res

res

CPU times: user 7.14 ms, sys: 824 μs, total: 7.96 ms
Wall time: 7.09 ms


9194

## Animation of the process

In [14]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

data = get_data()

mask = data == "@"  # Work with boolean array directly

# Store each iteration
frames = [mask.copy()]

res = 0
round_res = 1
while round_res > 0:
    neighbor_count = convolve(mask.astype(int), kernel, mode='constant', cval=0)
    to_remove = mask & (neighbor_count < 4)
    round_res = np.sum(to_remove)
    
    if round_res > 0:
        mask[to_remove] = False
        frames.append(mask.copy())
    res += round_res

# Create animation
fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(frames[0], cmap='viridis', interpolation='nearest')
ax.set_title('Iteration: 0')

def update(frame_idx):
    im.set_array(frames[frame_idx])
    ax.set_title(f'Iteration: {frame_idx}')
    return [im]

ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=200, blit=True)
plt.close()
HTML(ani.to_jshtml())